
<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: n-gram models </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

## Fenosoa Randrinjatovo

In [463]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [464]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile
    
    Returns:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [465]:

print("load training set..")
print("\n")
train_data, vocab = load_data("data/train1.txt")
print(train_data[0])
print(train_data[34])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("data/valid1.txt")


load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']
['<s>', 'the', 'aggressive', 'salesman', 'urged', 'me', 'to', 'sign', 'the', 'contract', 'right', 'away.', '</s>']


how : 107
load validation set


In [466]:
print("fathers :",vocab['my'])

fathers : 203


In [467]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count 
    
    Returns: 
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab 
    # or have a count that is below mincount
    ## FILL CODE
    data_with_unk = []
    for sentences in data:   
        for index , word in enumerate(sentences):            
            if vocab[word] <= mincount :
                sentences[index]="<unk>"
        data_with_unk.append(sentences)
        

    return data_with_unk

In [468]:
print("Before removing  rare words")
print(len(train_data))
print(train_data[0])
print(train_data[34])
print(train_data[-6])
print(" ")
print("="*125)
print("After removing rare words")

train_data = remove_rare_words(train_data, vocab, mincount = 1)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
#train_data

print(train_data[0])
print(train_data[34])
print(train_data[-6])
print(" ")
print(len(train_data))


Before removing  rare words
5561
['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']
['<s>', 'the', 'aggressive', 'salesman', 'urged', 'me', 'to', 'sign', 'the', 'contract', 'right', 'away.', '</s>']
['<s>', '"who', 'shot', 'him,', 'then?"', '"the', 'burglar,', 'sir.', '"', '</s>']
 
After removing rare words
['<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>']
['<s>', 'the', 'aggressive', '<unk>', 'urged', 'me', 'to', 'sign', 'the', '<unk>', 'right', 'away.', '</s>']
['<s>', '<unk>', 'shot', 'him,', 'then?"', '"the', '<unk>', '<unk>', '<unk>', '</s>']
 
5561


In [469]:
print(train_data[-6])

['<s>', '<unk>', 'shot', 'him,', 'then?"', '"the', '<unk>', '<unk>', '<unk>', '</s>']


In [470]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    n (int): size of the n-gram
    Returns:
    prob (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    '''
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    for sentence in data:
        sentence = tuple(sentence)
        ## FILL CODE
        # dict can be indexed by tuples
        # store in the same dict all the ngrams
        # by using the context as a key and the word as a value
        for index, _ in enumerate(sentence):
#             if index==4:
#                 counts[sentence[index-1:index]][sentence[index]]+=1
#                 return counts
#                 return   sentence, counts
            for N in range(n):
                if index >= N:
                    counts[sentence[index-N:index]][sentence[index]] += 1.0
                else:
                    break   


    prob = defaultdict(lambda: defaultdict(lambda: 0.0))
    # Build the probabilities from the counts
    # Be careful with how you normalize!
    for context in counts.keys():  
    ## FILL CODE
        den = 0
        for word in counts[context].keys():
            den += counts[context][word]
        for word in counts[context].keys():
            prob[context][word] = counts[context][word] / den
        
        
    return prob

In [471]:
# RUN TO BUILD NGRAM MODEL

n = 2
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)

print(model[()]["father"])

build ngram model with n =  2
0.0008031885068271023


Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [472]:
def get_prob(model, context, w):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context
    
    Retunrs:
    prob(float): probability of this word given the context 
    '''
    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model
    
    ## FILL CODE
    if w in model[context] and context in model:
        return model[context][w]
    else:
        return 0.4 * get_prob(model, context[1:], w)

In [473]:
def perplexity(model, data, n):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram
    
    Retunrs:
    perp(float): the perplexity of the model 
    '''
    ## FILL CODE
    perp, T = 0.0, 0
    for sentence in data:
        sentence = tuple(sentence)
        for i in range(1, len(sentence)): 
            index =  1-n if n-i<1 else  -i  
            context = sentence[i+index:i]
            perp += - np.log(get_prob(model, context, sentence[i]))
            T += 1
    return np.exp(perp / T)

In [474]:
# COMPUTE PERPLEXITY ON VALIDATION SET
# 53.582886955857006
print("The perplexity is", perplexity(model, valid_data, n=2))

The perplexity is 53.582886955857006


In [475]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context 
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and 
    thier probabilites
    
    Retunrs:
    words_and_probs(dic): {word: probability of word given context}
    
    This is a cat and dog
    
    '''
    # code a recursive function over context
    # to find the longest available ngram
    ## FILL CODE
    
    if not context in model:
        words_and_probs = get_proba_distrib(model, context[1:])
    else:
        words_and_probs = model[context]
    
    return words_and_probs

In [476]:
def generate(model):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model. 
    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    sentence = ["<s>"]
    while sentence[-1] != "</s>" and len(sentence)<100:
        ## FILL CODE
        proba = get_proba_distrib(model, tuple(sentence))
        w = np.random.choice(list(proba.keys()), 1, p = list(proba.values()))
        sentence.append(w[0])
    return sentence

In [477]:
# GENERATE A SENTENCE FROM THE MODEL
print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', 'tom', 'wants', 'somebody', 'to', 'meet', 'him', 'a', '<unk>', '<unk>', 'and', 'say', 'that', 'he', 'was', '<unk>', 'pacing', 'the', "world's", 'population', 'of', 'the', '<unk>', '</s>']


Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.

In [478]:
print("load training set..")
print("\n")
train_data2, vocab2 = load_data("data/train2.txt")
print(train_data2[0])
print(train_data2[34])
print("\n")
print("how :",vocab2['how'])
print("load validation set")
valid_data2, _ = load_data("data/valid2.txt")

load training set..


['<s>', 'i', 'liked', 'your', 'idea', 'and', 'adopted', 'it', '.', '</s>']
['<s>', 'those', 'were', 'his', '<unk>', 'words', '.', '</s>']


how : 3195
load validation set


In [479]:
print("Before removing  rare words")
print(len(train_data2))
print(train_data2[0])
print(train_data2[34])
print(train_data2[-6])
print(" ")
print("="*125)
print("After removing rare words")

train_data2 = remove_rare_words(train_data2, vocab, mincount = 1)
valid_data2 = remove_rare_words(valid_data2, vocab, mincount = 1)
#train_data

print(train_data2[0])
print(train_data2[34])
print(train_data2[-6])
print(" ")
print(len(train_data2))

Before removing  rare words
180000
['<s>', 'i', 'liked', 'your', 'idea', 'and', 'adopted', 'it', '.', '</s>']
['<s>', 'those', 'were', 'his', '<unk>', 'words', '.', '</s>']
['<s>', 'this', 'store', 'always', '<unk>', 'at', 'eight', '.', '</s>']
 
After removing rare words
['<s>', 'i', 'liked', 'your', 'idea', 'and', '<unk>', 'it', '<unk>', '</s>']
['<s>', 'those', 'were', 'his', '<unk>', 'words', '<unk>', '</s>']
['<s>', 'this', 'store', 'always', '<unk>', 'at', '<unk>', '<unk>', '</s>']
 
180000


In [480]:
# RUN TO BUILD NGRAM MODEL

n = 2
print("build ngram model with n = ", n)
model2 = build_ngram(train_data2, n)

print(model2[()]["father"])

build ngram model with n =  2
0.00077467423678778


In [493]:
print("")
print("The perplexity is", perplexity(model2, valid_data2, n=2))
print("")
print("Generated sentence: ",generate(model2))


The perplexity is 32.49482429320278

Generated sentence:  ['<s>', 'you', 'go', 'to', 'play', 'by', 'phone', '<unk>', 'was', 'expected', '<unk>', 'in', '<unk>', '<unk>', '</s>']
